#### Author: Madhusudhanan Balasubramanian (MB), Ph.D., The University of Memphis
#### Visualize model outputs

In [ ]:
#Libraries / modules
import os
import sys
import numpy as np

# Import Mask RCNN
ROOT_DIR = "./Mask_RCNN"
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
#
from keras.preprocessing.image import img_to_array
#

#Using pycocotools for AP and AR calculations as in centermask
import json
from pycocotools.coco import COCO
import pycocotools.mask as mask_util
import axon_coco as coco #copied samples/coco/coco.py as axon_coco.py
from axonlib.cocoeval import COCOeval
import pandas as pd
#
#import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

#Visualization
from axonlib.axon_visualizer_mrcnn import SimpleMetadata, Visualizer, ColorMode

Using TensorFlow backend.


In [ ]:
class InferenceParams():
    def __init__(self, model_root_dir, model_subdir, model_weight_file,
                 data_root_dir, dataset_name,
                 mrcnn_lib_dir=None):
        
        debug_flag = 0

        # Root directory of the project
        if mrcnn_lib_dir is None:
            self.mrcnn_lib_dir = "./Mask_RCNN"
        else:
            self.mrcnn_lib_dir = mrcnn_lib_dir

        #Model configuration
        self.inference_config = InferenceConfig()
        if debug_flag == 1:
            self.inference_config.display()

        #Model params
        self.model_root_dir = model_root_dir
        self.model_path = os.path.join(model_root_dir, model_subdir)
        self.model_weight_file = model_weight_file
        self.model_abs_file = os.path.join(self.model_path, self.model_weight_file)

        #Data params and initialization
        self.data_root_dir = data_root_dir
        self.dataset_name = dataset_name
        self.data_dir = os.path.join(self.data_root_dir, self.dataset_name)
        #
        self.axon_dataset = coco.CocoDataset()
        print(f"self.data_root_dir: {self.data_root_dir}")
        self.input_coco_obj = self.axon_dataset.load_coco(self.data_root_dir, self.dataset_name, return_coco=True)
        self.axon_dataset.prepare() # Must call before using the dataset

        #Evaluation / output directory
        self.eval_dir = os.path.join(self.model_path, 'Evaluations', dataset_name)
        os.makedirs(self.eval_dir, exist_ok=True)
        
        ft_json_file = f"{dataset_name}.json"
        self.gt_json_file = os.path.join(self.data_root_dir, self.dataset_name, ft_json_file)
        self.predictions_file = os.path.join(self.eval_dir, "coco_instances_results.json")


#Model configuration for inference
class InferenceConfig(coco.CocoConfig):
    #Dec 09, 2021 MB notes: Initially, no other configuration changes needed for training (recall / see that
    # configuration changes required for inferences such as setting # GPUs to 1, etc. See axon_coco.py for 
    # other possible configuration changes
    
    #General model parameters
    BACKBONE = 'resnet50' #default is resnet101
    USE_MINI_MASK=True
    MEAN_PIXEL = [123.7, 116.8, 103.9]
        
    #General training parameters
    #----------------
    IMAGES_PER_GPU = 1 #should be 1 for inference
    GPU_COUNT = 1
    BATCH_SIZE = IMAGES_PER_GPU * GPU_COUNT #BATCH_SIZE calculated only in config.py's constructor in line 216
    #
    STEPS_PER_EPOCH = 3 #Jan 07: reduced from 100 to 75; May 10, 2022: steps/epoch = #training images/batch size
    VALIDATION_STEPS = 3 #originally 5, previously set at 15
    #
    GRADIENT_CLIP_NORM = 10.0
    LEARNING_MOMENTUM = 0.9
    WEIGHT_DECAY=0.0001
    LOSS_WEIGHTS = { "rpn_class_loss": 1., "rpn_bbox_loss": 1., "mrcnn_class_loss": 1., "mrcnn_bbox_loss": 1., "mrcnn_mask_loss": 1. }

    #RPN parameters
    #---------------
    # Length of square anchor side in pixels
    #RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128) #Was used in V11 which generated the model e608
    #RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64) #Incorrectly set this in V11_Phase2 - that's why the model had issues
    # Ratios of anchors at each cell (width/height)
    # A value of 1 represents a square anchor, and 0.5 is a wide anchor
    #RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    RPN_ANCHOR_RATIOS = [0.25, 0.5, 1, 2, 4]
    #References for increasing number of detections: https://github.com/matterport/Mask_RCNN/issues/1884#:~:text=What%20seems%20to%20have%20had%20the%20greatest%20impact%20for%20us%20were%20the%20training%20configs%3A
    RPN_TRAIN_ANCHORS_PER_IMAGE = 600 #default: 256; Number of anchors per image selected to train the RPN
    MAX_GT_INSTANCES = 600 #Number of GT instances per image kept to train the network
    
    #Proposal layer parameters (not trainable layer; just filtering)
    #-------------------------
    PRE_NMS_LIMIT = 6000 #default: ;Number of anchors with the best RPN score that are retained
    RPN_NMS_THRESHOLD = 0.9 #0.7 #default is 0.7; area overlap among candidate anchors before dropping the anchor with the lowest score; higher values increases the number of region proposals
    POST_NMS_ROIS_TRAINING = 1800 # >Training; ROIs kept after non-maximum supression in the proposal layer; default is 2000
    POST_NMS_ROIS_INFERENCE = 8000 # >Inference; ROIs kept after NMS in the proposal layer based on their RPN score
    
    #Detection target layer parameters (training only; not a trainable layer; just filtering)
    #Receives at most POST_NMS_ROIS_TRAINING number of anchors from the proposal layer
    #Anchors whose IoU > 0.5 over ground truth are selected (at most POST_NMS_ROIS_TRAINING)
    #---------------------------------
    TRAIN_ROIS_PER_IMAGE = 600 #default is 200; no. of ROIs randomly selected out of POST_NMS_ROIS_TRAINING
    ROI_POSITIVE_RATIO = 0.33 #default is 0.33; usually set as 1/#classes; i.e. 33% of TRAIN_ROIS_PER_IMAGE should be positive
       
    #Detection layer parameters (inference only; not trainable, just filtering)
    #Receives at most POST_NMS_ROIS_INFERENCE number of anchors from the proposal layer
    #MB: https://medium.com/@umdfirecoml/training-a-mask-r-cnn-model-using-the-nucleus-data-bcb5fdbc0181 
    #----------------------------
    DETECTION_MIN_CONFIDENCE = 0.7 #default: 0.7; AOIs with lower confidence than this are discarded
    DETECTION_NMS_THRESHOLD = 0.3 #reference: 0.3; AOIs those with higher overlapping areas are discarded based on their RPN score

    #Feature Pyramid Network (FPN)
    #Has a classifier and mask graph; identifies class and generates mask
    #---------------------------------
    DETECTION_MAX_INSTANCES = 600 # >Inference; maximum number of instances identified by Mask RCNN

In [3]:
def save_GT_annotations(meta_data, image_mat, data_dict, output_image_file):
    """
        Save visualization of GT object annotations
    """
    
    gt_outline_linewidth=0.8
    gt_outline_linestyle='-'

    #Instantiate Visualizer object "v" with the image and the metadata
    viz_obj = Visualizer(image_mat[:, :, ::-1],
                   metadata = meta_data, 
                   #MB: March 31, 2024
                   #instance_mode=ColorMode.IMAGE_BW,   # remove the colors of unsegmented pixels. This option is only available for segmentation models
                   instance_mode=ColorMode.SEGMENTATION
    )
    #
    #Draw the ground truth first
    visImage_obj = viz_obj.draw_dataset_dict(data_dict, jittering=False, outline_linewidth=gt_outline_linewidth, outline_linestyle=gt_outline_linestyle)
    output_image = visImage_obj.get_image()[:, :, ::-1]
    cv2.imwrite(output_image_file, output_image[:, :, ::-1])
    #
    #visImage_obj.save(output_image_file)

def save_detected_annotations(meta_data, image_mat, detected_instances, output_image_file):
    """
        Save visualization of annotations of objects detected by the model
    """
    
    dt_outline_linewidth = 0.8
    dt_outline_linestyle = '-'

    #Instantiate Visualizer object "v" with the image and the metadata
    viz_obj = Visualizer(image_mat[:, :, ::-1],
                   metadata = meta_data, 
                   #MB: March 31, 2024
                   #instance_mode=ColorMode.IMAGE_BW,   # remove the colors of unsegmented pixels. This option is only available for segmentation models
                   instance_mode=ColorMode.SEGMENTATION
    )
    #
    #Draw the detected annotations on top of the input image
    visImage_obj = viz_obj.draw_instance_predictions(detected_instances, jittering=False, outline_linewidth=dt_outline_linewidth)
    output_image = visImage_obj.get_image()[:, :, ::-1]
    cv2.imwrite(output_image_file, output_image)
    #
    #visImage_obj.save(output_image_file)

def save_GT_DT_combined_annotations(meta_data, image_mat, data_dict, detected_instances, output_image_file):
    """
        Save combined annotation of GT objects and objects detected by the model
    """

    gt_outline_linewidth = 0.8
    gt_outline_linestyle = ':'
    dt_outline_linewidth = 0.4

    #GT image
    #
    #Instantiate Visualizer object "v" with the image and the metadata
    viz_obj = Visualizer(image_mat[:, :, ::-1],
                   metadata = meta_data, 
                   #MB: March 31, 2024
                   #instance_mode=ColorMode.IMAGE_BW,   # remove the colors of unsegmented pixels. This option is only available for segmentation models
                   instance_mode=ColorMode.SEGMENTATION
    )
    #
    #Draw the ground truth first
    visImage_obj = viz_obj.draw_dataset_dict(data_dict, jittering=False, outline_linewidth=gt_outline_linewidth, outline_linestyle=gt_outline_linestyle)

    #Draw model detections on top of the ground truth
    visImage_obj = viz_obj.draw_instance_predictions(detected_instances, jittering=False, outline_linewidth=dt_outline_linewidth)

    # Get the drawing as image
    out_image_combined = visImage_obj.get_image()[:, :, ::-1]
    cv2.imwrite(output_image_file, out_image_combined[:, :, ::-1])
    #
    #visImage_obj.save(output_image_file)

In [4]:
def save_instance_predictions_image(inf_params):
    
    #Initialize
    input_coco_obj = inf_params.input_coco_obj
    axon_dataset = inf_params.axon_dataset
    curr_meta_data = SimpleMetadata(thing_classes=['Necrotic', 'Healthy'], 
                                    thing_colors = [(0, 0, 255), (255, 0, 0), (0, 255, 0)])

    #Create a Mask RCNN model in "inference" mode
    model = modellib.MaskRCNN(mode="inference", 
                              config=inf_params.inference_config, 
                              model_dir = inf_params.model_root_dir)

    #Load model file
    model.load_weights(inf_params.model_abs_file, by_name=True)

    image_ids = []
    all_results = []
    category_ids = input_coco_obj.getCatIds()
    #print(f"category_ids: {category_ids}")
    #print(f"image ids: {axon_dataset.image_ids}")
    #print(f"image ids: {input_coco_obj.getImgIds()}")
    all_image_ids = input_coco_obj.getImgIds()
    for image_ind, image_id in enumerate(all_image_ids):
        
        #MB, important: load_image_gt is scaling the image to 1024 x 1024 before feeding to detect().
        # But detect() has internal mechanisms to scale/mold and unscale/unmold.  This is done based on the 
        # size of the input image and the config parameters IMAGE_MIN_SIZE, IMAGE_MAX_SIZE.  With load_image_gt(),
        # and with IMAGE_MAX_SIZE = 1024, the output from detect() is not unmolded.  i.e. detections won't match
        # with the original gt geometry of the annotations - replacing with simple imread instead
        #
        #original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        #    modellib.load_image_gt(axon_dataset, inf_params.inference_config, image_ind, use_mini_mask=False)
        #MB, problem reading from axon_dataset.image_info, use coco obj instead
        #print(f"image_id:{image_id}; ind: {image_ind}; path:{axon_dataset.image_info[image_ind]['path']}")
        #original_image = cv2.imread(axon_dataset.image_info[image_ind]['path'])
        img_file_name = os.path.join(inf_params.data_dir, input_coco_obj.imgs[image_id]['file_name'])
        print(f"image_id:{image_id}; ind: {image_ind}; img_file_name: {img_file_name}")
        original_image = cv2.imread(img_file_name)
        
        #MB note: it appears mold_image normalizes the image (subtract dataset mean)
        #         img_to_array(image) is used in test_axon_model_v1.ipynb
        #         This needs to be verified:
        #           detect() molds all input images in a format for feeding to the network
        #           So, no need to mold before calling detect
        #           expand_dims adds a new axis to represent multiple images fed to the network for detection
        #           np.expand_dims(image, 0) is same as [image]--the latter is a list with single entry
        #scaled_image = modellib.mold_image(image, cfg)
        #image_array = np.expand_dims(scaled_image, 0)
        image_array = img_to_array(original_image)
        #print(f"image_array.shape: {image_array.shape}; dtype: {image_array.dtype}; max: {image_array.max()}; min: {image_array.min()}")
        #plt.imshow(image_array.astype(np.uint8))
        #plt.show()

        #Detect objects in the image
        result = model.detect([image_array])[0]
        #
        #boxes = result["rois"]
        #masks = result["masks"]
        #class_ids = result["class_ids"]
        #class_names = axon_dataset.class_names
        #scores = result["scores"]

        DT_image_file = os.path.join(inf_params.eval_dir, input_coco_obj.imgs[image_id]['file_name'])
        save_detected_annotations(curr_meta_data, image_array, result, DT_image_file)

In [ ]:
model_root_dir = "./logs/"
model_subdir = "coco20250418T0934"
model_weight_file = "mask_rcnn_coco_0263.h5"
#
# Data directory
data_root_dir = "./DataFiles/Phase3"

In [6]:
dataset_name = "Phase3_Validation"
bbox_or_segm = "segm"

inf_params = InferenceParams(model_root_dir, model_subdir, model_weight_file,
                             data_root_dir, dataset_name, mrcnn_lib_dir=None)

#Evaluate the model with the given dataset in dataset_name
save_instance_predictions_image(inf_params)

self.data_root_dir: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.






Re-starting from epoch 263
image_id:5167; ind: 0; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Validation/2006---Rat-77E6772---p88-F_m10_06_A_D.tif
image_id:5168; ind: 1; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Validation/2006---Rat-77E7654---p87-M--NEW_m10_11_D_A.tif
image_id:5169; ind: 2; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Validation/2006---Rat-77E7654---p87-M--NEW_m16_07_B_C.tif
image_id:5170; ind: 3; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Validation/2006---Rat-77E82F7---p92-M-

In [7]:
dataset_name = "Phase3_Testing"
bbox_or_segm = "segm"

inf_params = InferenceParams(model_root_dir, model_subdir, model_weight_file,
                             data_root_dir, dataset_name, mrcnn_lib_dir=None)

#Evaluate the model with the given dataset in dataset_name
save_instance_predictions_image(inf_params)

self.data_root_dir: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Re-starting from epoch 263
image_id:5181; ind: 0; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Testing/2006---Rat-77E6772---p88-F_m15_09_C_A.tif
image_id:5182; ind: 1; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Testing/2006---Rat-77E6772---p88-F_m16_05_A_C.tif
image_id:5183; ind: 2; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Testing/2006---Rat-77E7654---p87-M--NEW_m19_06_A_D.tif
image_id:5184; ind: 3; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Testing/2006---Rat-77E7654---p87-M--NEW_m23_08_B_D.tif
image_id:5185; ind: 4; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Phase3_Testing/2006---Rat-77E7975---p86-F--NEW_m02_13_C_C.tif
image_id:5186; ind: 5; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles

In [8]:
dataset_name = "Quigley_Eval"
bbox_or_segm = "segm"

inf_params = InferenceParams(model_root_dir, model_subdir, model_weight_file,
                             data_root_dir, dataset_name, mrcnn_lib_dir=None)

#Evaluate the model with the given dataset in dataset_name
save_instance_predictions_image(inf_params)

self.data_root_dir: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
Re-starting from epoch 263
image_id:5254; ind: 0; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval/Ms6770ON01_1x1.tif
image_id:5255; ind: 1; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval/Ms6770ON01_1x2.tif
image_id:5256; ind: 2; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval/Ms6770ON01_2x1.tif
image_id:5257; ind: 3; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval/Ms6770ON01_2x2.tif
image_id:5258; ind: 4; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval/Ms6770ON02_1x1.tif
image_id:5259; ind: 5; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval/Ms6770ON02_1x2.tif
image_id:5260; ind: 6; img_file_name: /home/madhu/Lab/Members/00_madhu/DataFiles/Phase3/Quigley_Eval